# 世界の最新の感染者数

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
del df["Lat"]
del df["Long"]

In [3]:
df.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,84050,85892,87716,88740,89861,91458,93272,93288,96531,98734
1,NaN,Albania,0,0,0,0,0,0,0,0,...,132415,132426,132437,132449,132459,132461,132469,132476,132481,132484
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,132034,132355,132727,133070,133388,133742,134115,134458,134840,135219
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,13791,13805,13813,13813,13813,13826,13828,13836,13839,13842
4,NaN,Angola,0,0,0,0,0,0,0,0,...,36115,36325,36455,36600,36705,36790,36921,37094,37289,37467


In [4]:
latest_date = df.tail(1).columns[-1]
latest_date

'6/18/21'

In [5]:
latest_column_key = df.columns[-1]
latest_column_key2 = df.columns[-2]
df = df[['Province/State','Country/Region',latest_column_key2,latest_column_key]]
df = df.rename(columns={latest_column_key:"latest_confirmed"})
df = df.rename(columns={latest_column_key2:"latest_confirmed2"})
df["new_confirmed"] = df.latest_confirmed - df.latest_confirmed2
df.head()

,Province/State,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
0,NaN,Afghanistan,96531,98734,2203
1,NaN,Albania,132481,132484,3
2,NaN,Algeria,134840,135219,379
3,NaN,Andorra,13839,13842,3
4,NaN,Angola,37289,37467,178


## 全世界の最新の感染者数

In [6]:
print("全世界感染者数({}時点):{:,}人".format(latest_date,df["latest_confirmed"].sum()))
print("全世界感染者数平均({}時点):{:,}人".format(latest_date,df["latest_confirmed"].mean()))
print("全世界の新規感染者数({}時点):{:,}人".format(latest_date,df["new_confirmed"].sum()))
print("全世界の新規感染者数平均({}時点):{:,}人".format(latest_date,df["new_confirmed"].mean()))


全世界感染者数(6/18/21時点):177,764,675人
全世界感染者数平均(6/18/21時点):639,441.2769784173人
全世界の新規感染者数(6/18/21時点):330,078人
全世界の新規感染者数平均(6/18/21時点):1,187.3309352517986人


## 国別の最新の感染者数

In [7]:
country = df.groupby(["Country/Region"],as_index=False).sum("latest_confirmed")
country = country.sort_values("latest_confirmed", ascending=False)

In [8]:
country.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
180,US,33508867,33516175,7308
79,India,29762793,29762793,0
23,Brazil,17702630,17801462,98832
62,France,5811461,5814648,3187
179,Turkey,5354153,5359728,5575


In [9]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [10]:
plt.figure(figsize=(9.5,5))
plt.title("Worst 50")
plt.xticks(rotation="90",fontsize=9)
plt.ylabel("count (1000 people)")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.bar(country.head(50)["Country/Region"],country.head(50)["latest_confirmed"]/1000,label="positive")
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

## ヒストグラム 

In [11]:
plt.figure(figsize=(10,5))
plt.title("histgram(million)")
#plt.xticks(rotation="90",fontsize=9)
#plt.ylabel("count (1000 people)")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.hist(country["latest_confirmed"]/1000000,bins=20)

<IPython.core.display.Javascript object>

(array([174.,   8.,   6.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.]),
 array([1.00000000e-06, 1.67580970e+00, 3.35161840e+00, 5.02742710e+00,
        6.70323580e+00, 8.37904450e+00, 1.00548532e+01, 1.17306619e+01,
        1.34064706e+01, 1.50822793e+01, 1.67580880e+01, 1.84338967e+01,
        2.01097054e+01, 2.17855141e+01, 2.34613228e+01, 2.51371315e+01,
        2.68129402e+01, 2.84887489e+01, 3.01645576e+01, 3.18403663e+01,
        3.35161750e+01]),
 <a list of 20 Patch objects>)

## 人口当たりの分析

In [12]:
pop = pd.read_excel("world-data/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx",header=16)
pop.head(1)

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950,1951,1952,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Estimates,WORLD,NaN,900,World,0,2536431.018,2584034.227,2630861.69,...,7041194.168,7125827.957,7210582.041,7295290.759,7379796.967,7464021.934,7547858.9,7631091.113,7713468.205,7794798.729


In [13]:
pop.iloc[:,-1]

0      7794798.729
1              ...
2      1273304.261
3      6521494.468
4      1057438.163
          ...     
284         62.273
285      37742.157
286         56.772
287          5.795
288     331002.647
Name: 2020, Length: 289, dtype: object

In [14]:
pop_latest = pop.loc[pop["Type"]=="Country/Area"][[pop.columns[2],pop.columns[-1]]]

In [15]:
pop_latest.columns = ["country","population"]

In [16]:
pop_latest.head()

,country,population
26,Burundi,11890.781
27,Comoros,869.595
28,Djibouti,988.002
29,Eritrea,3546.427
30,Ethiopia,114963.583


In [17]:
country.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
180,US,33508867,33516175,7308
79,India,29762793,29762793,0
23,Brazil,17702630,17801462,98832
62,France,5811461,5814648,3187
179,Turkey,5354153,5359728,5575


### 世界全体での人口10万人あたり陽性者数

In [18]:
country.latest_confirmed.sum()

177764675

In [19]:
pop_latest.population.sum()

7794798.729

In [20]:
print("****世界全体での人口10万人あたり陽性者数:{}****".format(country.latest_confirmed.sum() / (pop_latest.population.sum() / 100)))

****世界全体での人口10万人あたり陽性者数:2280.5550365096537****


### 各国の人口１０万人あたり陽性者数

In [21]:
c_merged = pd.merge(country, pop_latest, left_on="Country/Region", right_on="country", how="left")
c_merged.loc[c_merged["country"].isna()]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population
0,US,33508867,33516175,7308,NaN,NaN
5,Russia,5203117,5220168,17051,NaN,NaN
12,Iran,3070426,3080526,10100,NaN,NaN
48,Bolivia,417195,417195,0,NaN,NaN
61,West Bank and Gaza,312334,312499,165,NaN,NaN
69,Venezuela,256862,258140,1278,NaN,NaN
71,Moldova,255994,256054,60,NaN,NaN
84,"Korea, South",150238,150720,482,NaN,NaN
85,Burma,146768,147069,301,NaN,NaN
93,Kosovo,107192,107192,0,NaN,NaN


In [22]:
p_merged = pd.merge(pop_latest, country, left_on="country", right_on="Country/Region", how="left")
len(p_merged.loc[p_merged["Country/Region"].isna()])

63

In [23]:
pop_latest.loc[pop_latest["country"].str.contains("Syria")]

,country,population
104,Syrian Arab Republic,17500.657


In [24]:
renames = {
    "US":"United States of America",
    "Russia":"Russian Federation",
    "Iran":"Iran (Islamic Republic of)",
    "Bolivia":"Bolivia (Plurinational State of)",
    "Moldova":"Republic of Moldova",
    "Venezuela":"Venezuela (Bolivarian Republic of)",
    "Korea, South":"Republic of Korea",
    "Congo (Kinshasa)":"Democratic Republic of the Congo",
    "Congo (Brazzaville)":"Congo",
    "Vietnam":"Viet Nam",
    "Laos":"Lao People's Democratic Republic",
    "Taiwan*":"China, Taiwan Province of China",
    "Tanzania":"United Republic of Tanzania",
    "Brunei":"Brunei Darussalam",
    "Micronesia":"Micronesia (Fed. States of)",
    "Syria":"Syrian Arab Republic",
}

In [25]:
country_renamed = country.copy()
country_renamed["Country/Region"].replace(renames,inplace=True)

In [26]:
c_merged = pd.merge(country_renamed, pop_latest, left_on="Country/Region", right_on="country", how="left")
c_merged.loc[c_merged["country"].isna()]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population
61,West Bank and Gaza,312334,312499,165,NaN,NaN
85,Burma,146768,147069,301,NaN,NaN
93,Kosovo,107192,107192,0,NaN,NaN
112,Cote d'Ivoire,47973,47973,0,NaN,NaN
179,Diamond Princess,712,712,0,NaN,NaN
187,MS Zaandam,9,9,0,NaN,NaN


In [27]:
c_merged["positives per 1000 population"] = c_merged["latest_confirmed"] / (c_merged["population"])
c_merged["positives per 100,000 population"] = (c_merged["latest_confirmed"] / (c_merged["population"]/100)).astype(float)
c_merged["new positives per 100,000 population"] = (c_merged["new_confirmed"] / (c_merged["population"]/100)).astype(float)

In [28]:
c_merged = c_merged.sort_values("positives per 1000 population", ascending=False)

In [29]:
c_merged.loc[c_merged["Country/Region"]=="United States of America"]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population,positives per 1000 population,"positives per 100,000 population","new positives per 100,000 population"
0,United States of America,33508867,33516175,7308,United States of America,331002.647,101.256517,10125.651654,2.207837


In [30]:
fig = plt.figure(figsize=(12,8))

plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("positives per pupulation WORST 50")
#plt.xticks(rotation="90",fontsize=9)
plt.yticks(fontsize=9)
plt.xlabel("count per 100,000 population")
plt.grid(axis="x")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
#plt.bar(c_merged.head(50)["Country/Region"],c_merged.head(50)["positives per 100,000 population"],label="positive")
chart_data=c_merged.head(50).sort_values("positives per 100,000 population",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["positives per 100,000 population"],label="positive")
#plt.text(0,chart_data["positives per 100,000 population"],chart_data["population"].astype(str))
#plt.text(0,chart_data["positives per 100,000 population"].all(),chart_data["population"].astype(str).all())
plt.legend()

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("positives per pupulation BEST 50")
#plt.xticks(rotation="90",fontsize=9)
plt.yticks(fontsize=9)
plt.xlabel("count per 100,000 population")
plt.grid(axis="x")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
c_asc = c_merged.sort_values("positives per 1000 population")
chart_data=c_asc.head(50).sort_values("positives per 100,000 population",ascending=False)
plt.barh(chart_data["Country/Region"],chart_data["positives per 100,000 population"],label="positive")
plt.legend()

fig.tight_layout()


<IPython.core.display.Javascript object>

In [31]:
fig.savefig("out/covid-global-tops.png")

In [32]:
plt.figure(figsize=(9.5,5))
plt.title("histgram(positives per 100,000 pop)")
plt.hist(c_merged["positives per 100,000 population"],bins=[0,2000,4000,8000,10000,12000,14000,16000,18000])

<IPython.core.display.Javascript object>

(array([97., 27., 36., 15.,  5.,  2.,  5.,  1.]),
 array([    0,  2000,  4000,  8000, 10000, 12000, 14000, 16000, 18000]),
 <a list of 8 Patch objects>)

In [33]:
c_merged.describe()

,latest_confirmed2,latest_confirmed,new_confirmed,"positives per 100,000 population","new positives per 100,000 population"
count,1.940000e+02,1.940000e+02,194.000000,188.000000,188.000000
mean,9.146113e+05,9.163128e+05,1701.432990,3507.914837,6.189663
std,3.540143e+06,3.543441e+06,7767.121687,4039.496772,12.369624
min,1.000000e+00,1.000000e+00,0.000000,0.852108,0.000000
25%,1.225600e+04,1.235050e+04,1.250000,228.781823,0.043265
50%,9.827550e+04,9.938500e+04,78.500000,1827.842011,1.310334
75%,4.150515e+05,4.152425e+05,798.250000,6349.335666,5.823366
max,3.350887e+07,3.351618e+07,98832.000000,17914.967967,80.377221


In [34]:
c_merged.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population,positives per 1000 population,"positives per 100,000 population","new positives per 100,000 population"
138,Andorra,13839,13842,3,Andorra,77.265,179.14968,17914.967967,3.882741
96,Montenegro,100020,100036,16,Montenegro,628.062,159.277269,15927.726880,2.547519
20,Czechia,1665660,1665818,158,Czechia,10708.982,155.553348,15555.334765,1.475397
68,Bahrain,261501,261988,487,Bahrain,1701.583,153.967218,15396.721759,28.620408
162,San Marino,5090,5090,0,San Marino,33.938,149.979374,14997.937415,0.000000


In [35]:
c_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 138 to 187
Data columns (total 9 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Country/Region                        194 non-null    object 
 1   latest_confirmed2                     194 non-null    int64  
 2   latest_confirmed                      194 non-null    int64  
 3   new_confirmed                         194 non-null    int64  
 4   country                               188 non-null    object 
 5   population                            188 non-null    object 
 6   positives per 1000 population         188 non-null    object 
 7   positives per 100,000 population      188 non-null    float64
 8   new positives per 100,000 population  188 non-null    float64
dtypes: float64(2), int64(3), object(4)
memory usage: 15.2+ KB


In [36]:
plt.figure(figsize=(9.5,5))
plt.title("histgram(positives per 100,000 pop)")
plt.hist(c_merged["positives per 100,000 population"],bins=200)

<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/masaru/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([26., 16., 14.,  9.,  2.,  3.,  3.,  2.,  1.,  0.,  1.,  1.,  3.,
         3.,  1.,  3.,  1.,  1.,  3.,  1.,  0.,  3.,  0.,  2.,  2.,  0.,
         2.,  1.,  2.,  0.,  3.,  2.,  1.,  1.,  1.,  2.,  2.,  2.,  0.,
         2.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,
         0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
         0.,  0.,  2.,  2.,  1.,  2.,  0.,  1.,  0.,  0.,  0.,  2.,  1.,
         1.,  1.,  2.,  1.,  0.,  1.,  2.,  2.,  0.,  2.,  2.,  1.,  1.,
         1.,  0.,  2.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,
         1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.

## 新規感染者数

In [37]:
fig = plt.figure(figsize=(12,8))

plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("新規感染者数 Worst 50({})".format(latest_date))
plt.yticks(fontsize=9)
plt.xlabel("count (people)")
chart_data = country.sort_values("new_confirmed",ascending=False).head(50).sort_values("new_confirmed",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["new_confirmed"],label="positive")
plt.grid(axis="x")
mean = int(country.new_confirmed.mean())
plt.plot([mean,mean],[0,50], 'k--', lw=0.5)
plt.text(mean,50,"平均:{}".format(mean),ha="center",fontsize=9)
plt.legend()

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("人口１０万人当たり新規感染者数 Worst 50({})".format(latest_date))
plt.yticks(fontsize=9)
plt.xlabel("count (people)")
chart_data = c_merged.sort_values("new positives per 100,000 population",ascending=False).head(50).sort_values("new positives per 100,000 population",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["new positives per 100,000 population"],label="positive")
plt.grid(axis="x")
mean = int(c_merged["new positives per 100,000 population"].mean())
plt.plot([mean,mean],[0,50], 'k--', lw=0.5)
plt.text(mean,50,"平均:{}".format(mean),ha="center",fontsize=9)
plt.legend()

fig.text(0,0
         ,"※感染者の情報提供:COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University,"\
         +"※人口の情報提供:WHO",
        fontsize=7,ha="left",va="bottom")
#fig.text(1,0.0,"※人口の情報提供:WHO",fontsize=8,ha="right")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [38]:
fig.savefig("out/covid-global-new-positives-latest-top.png")

### WORST50

In [39]:
country.sort_values("new_confirmed",ascending=False).head(50).reset_index()


,index,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
0,23,Brazil,17702630,17801462,98832
1,37,Colombia,3859824,3888614,28790
2,6,Argentina,4222400,4242763,20363
3,144,Russia,5203117,5220168,17051
4,80,Indonesia,1950276,1963266,12990
5,162,South Africa,1786079,1796589,10510
6,184,United Kingdom,4616628,4626915,10287
7,81,Iran,3070426,3080526,10100
8,180,US,33508867,33516175,7308
9,139,Philippines,1339457,1346276,6819
